# Quantum Imaginary Time Evolution (QITE)

**Table of Contents**

- [Introduction](#Introduction)
- [Hamiltonian and Initial State](#Hamiltonian-and-Initial-State)
- [QITE](#QITE)


## Introduction

In this tutorial, we will introduce how to simulate the imaginary-time evolution of quantum systems. The approach is grounded in a quantum algorithm that can efficiently model many-body quantum systems and approximate their ground state energies within polynomial time. Specifically, our focus lies on finding the ground state of a quantum system through Imaginary Time Evolution (ITE). This process is not only crucial for understanding quantum phase transitions but also plays a pivotal role in solving NP-hard problems such as the 3D Ising model and QMA-complete problems like those described by the Heisenberg model.

The imaginary-time evolution of quantum many-body systems is governed by the imaginary-time Schrödinger equation: 
$$
\partial_\tau|\phi(\tau)\rangle = -H|\phi(\tau)\rangle
,$$
where $\tau$ denotes imaginary time and $H$ is the time-independent Hamiltonian with an initial state $|\phi\rangle=|\phi(0)\rangle$. Our goal is to prepare a normalized imaginary-time evolved state 
$$
|\phi(\tau)\rangle=e^{-\tau H}|\phi\rangle / \|e^{-\tau H}|\phi\rangle\|
$$
on a quantum computer—a process known as ITE operation. For sufficiently large $\tau$, the system state typically converges to the ground state within the subspace induced by the initial state, often coinciding with the ground state of the Hamiltonian.

In [1]:
import numpy as np
import torch

import quairkit as qkit
from quairkit import to_state, Hamiltonian
from quairkit.database import *
from quairkit.qinfo import *

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from qsp import *
from qite import *

import warnings

revise_tol(1e-40)

qkit.set_dtype('complex128')

In [2]:
warnings.filterwarnings('ignore')

## Hamiltonian and Initial State

To streamline the analysis, we make the following assumptions regarding the system's Hamiltonian and initial state:

1. **Hamiltonian Form**: The Hamiltonian $H$ consists of a linear combination of Pauli operators, i.e., $H=\sum_j h_j\sigma_j$, where $h_j$ are real coefficients. By appropriately rescaling and shifting the Hamiltonian, we ensure its eigenvalues lie within the interval $[-1, 1]$, with the ground state energy $\lambda_0$ being negative.
2. **Non-zero Spectral Gap**: We assume there exists a strictly positive spectral gap $\Delta=\lambda_1-\lambda_0>0$, indicating a unique ground state.
3. **Sufficient Imaginary Time Parameter**: The imaginary time parameter $\tau$ must be sufficiently large to satisfy certain polynomial conditions such as $|\lambda_0| \cdot poly(\tau) \gg 1$, $|\gamma|^2 \cdot poly(\tau) \gg 1$, and $\Delta \cdot \tau \gg 1$, where $\gamma=\langle\psi_0|\phi\rangle$ represents the overlap between the initial state $|\phi\rangle$ and the ground state $|\psi_0\rangle$.
4. **Positive Overlap Parameter**: The overlap parameter $\gamma=\langle\psi_0|\phi\rangle$ is positive, ensuring that the initial state contains components of the ground state.
5. **Finite Copies of Initial State**: Access to a finite number of copies of the initial state $|\phi\rangle$ is assumed.

These assumptions ensure the feasibility and theoretical foundation of the algorithm while simplifying practical implementation complexities. In the following sections, we will detail how to leverage these assumptions and background knowledge to write relevant code for simulating imaginary-time evolution of quantum systems.

The hamiltonian we used in experiment is 
$$
H = \beta\sum_j-(X_jX_{j+1}+Y_jY_{j+1}+Z_jZ_{j+1})-\frac{\beta}{2}\sum_{j}^{}Z_j
$$

In [3]:
def hamiltonian_heisenberg(n=6, hz=1, hx=0, hy=0, hzz=1, hxx=1, hyy=1):
    pauli_terms = []

    # Single-qubit terms
    for i in range(n):
        if hx != 0:
            pauli_terms.append([hx, f'X{i}'])
        if hy != 0:
            pauli_terms.append([hy, f'Y{i}'])
        if hz != 0:
            pauli_terms.append([hz, f'Z{i}'])

    # Two-qubit terms (OBC)
    for i in range(n - 1):
        if hxx != 0:
            pauli_terms.append([hxx, f'X{i}, X{i + 1}'])
        if hyy != 0:
            pauli_terms.append([hyy, f'Y{i}, Y{i + 1}'])
        if hzz != 0:
            pauli_terms.append([hzz, f'Z{i}, Z{i + 1}'])

    return Hamiltonian(pauli_terms)

Get normalized Hamiltonian and initial state.

In [4]:
qb_num = 4

H_init = hamiltonian_heisenberg(n=qb_num, hx=-0.5, hz=0, hxx=-1, hyy=-1, hzz=-1)
max_abs_eigen = (torch.linalg.eigvalsh(H_init.matrix)).abs().max()

new_pauli_string = [[coef / max_abs_eigen, pauli_str]
                    for coef, pauli_str in H_init.pauli_str]
H = Hamiltonian(new_pauli_string)

phi = zero_state(qb_num)

In [5]:
eigenvalues, eigenvectors = torch.linalg.eigh(H.matrix)

min_eigen = eigenvalues.real.min()
min_index = torch.argmin(min_eigen)

min_eigenvector = eigenvectors[:, min_index]
print(f'min_eigenvector = {min_eigenvector}')
print(f'min_eigenvec = {min_eigen}')

min_eigenvector_state = to_state(min_eigenvector)

gamma = phi.bra @ min_eigenvector_state.ket
print(f'gamma = {gamma}')

min_eigenvector = tensor([-0.2500+0.j, -0.2500+0.j, -0.2500+0.j, -0.2500+0.j, -0.2500+0.j, -0.2500+0.j,
        -0.2500+0.j, -0.2500+0.j, -0.2500+0.j, -0.2500+0.j, -0.2500+0.j, -0.2500+0.j,
        -0.2500+0.j, -0.2500+0.j, -0.2500+0.j, -0.2500+0.j])
min_eigenvec = -0.7735026918962583
gamma = tensor([[-0.2500+0.j]])


In [6]:
sorted_indices = torch.argsort(eigenvalues.real)

smallest_indices = sorted_indices[:3]
smallest_eigenvalues = eigenvalues[smallest_indices]

smallest_eigenvector = eigenvectors[:, smallest_indices[0]]
second_smallest_eigenvector = eigenvectors[:, smallest_indices[1]]
third_smallest_eigenvector = eigenvectors[:, smallest_indices[2]]
smallest_eigenvector_state = to_state(smallest_eigenvector)
second_smallest_eigenvector_state = to_state(second_smallest_eigenvector)
third_smallest_eigenvector_state = to_state(third_smallest_eigenvector)

## QITE

Set initial data for QITE: $\tau$ and deg.

In [7]:
tau = 20
deg = 300

We have set up two interfaces. The first one is `only_P`; if it is set to `True`, the simulation of QITE will be directly carried out using the Laurent polynomial, otherwise, the QPP (Quantum Polynomial Protocol) will be used. The second interface is `learn`, which allows us to learn and further optimize the angles in QPP.

In [8]:
learn = False

Using QPE to get normalized lambda value within $1 / \tau$. Here, we directly provide this value and get the QPP angle. 

In [9]:
normalized_lambda = -min_eigen.item() + 1 / tau

list_theta, list_phi = get_qpp_angle(guess_lambda=normalized_lambda, tau=tau, deg=deg, learn=learn)

Computations of angles for QPP are completed with mean error 0.0018916676320003326


Using the oracle to get the QPP circuit. Here, we used the ideal evolution operator instead of Trotter form.

In [10]:
U = torch.matrix_exp(-1j * H.matrix)
cir = qpp_cir(list_theta, list_phi, U)

Considering the ancilla qubit, we have output state. The projection probability is the probability of measuring the ancilla qubit in the $|0\rangle$ state after applying the quantum circuit. In other words, it corresponds to the squared norm of the component of the output state where the ancilla is in the $|0\rangle$ state.

In [11]:
cir.collapse([0], post_selection=0, if_print=True)

In [12]:
input_state = nkron(zero_state(1), phi)
output_state = cir(input_state)

systems [0] collapse to the state |0> with (average) probability 0.0062952880666984476


The final normalized output state is obtained by dividing the output state by its norm.

In [13]:
fidelity = state_fidelity(output_state.trace(0), min_eigenvector_state)
print(f'fidelity = {fidelity}')

fidelity = 0.9962542188017928


---

In [14]:
qkit.print_info()


---------VERSION---------
quairkit: 0.4.0
torch: 2.7.0+cpu
numpy: 1.26.0
scipy: 1.14.1
matplotlib: 3.10.0
---------SYSTEM---------
Python version: 3.10.16
OS: Windows
OS version: 10.0.26100
---------DEVICE---------
CPU: ARMv8 (64-bit) Family 8 Model 1 Revision 201, Qualcomm Technologies Inc
